# **Lecture 14 - JavaScript Object Notation and APIs**

---

<br>

### Packages

In [ ]:
# install packages
install.packages("jsonlite")

In [ ]:
# load libraries
library(jsonlite)

<br>

<br>

---

<br>

### JavaScript Object Notation (JSON)

* Most datasets are stored in tabular format using `.csv`, `.txt`, or `.xlsx` file types

* `R` loads these datasets as a data frame or "structured list"

* However, structured lists are highly inefficent at storing hierarchical or nested data structures



<br>

* For example, the data below shows a very simple nested structure stored in tabular form

* To accommdate the varying number of hobbies, we must repeat information across other columns

* Storing data in this way can be inefficient in terms of data storage

In [ ]:
# load nested data in tabular format
read.csv("https://raw.githubusercontent.com/khasenst/datasets_teaching/refs/heads/main/nested_csv_example.csv",
          header = TRUE)

<br>

* A more efficient way of storing nested data is JavaScript Object Notation (JSON)

* JSON format can be thought of as an unstructured (nested) list, where each list item can contain another data structure
  * i.e. a list of lists, data frames, vectors or a mixture of these

<br>

* The script below shows an example of JSON formatted data

* List items are defined within curly braces `{}`

* The colon `:` is followed by a data element

* The square brackets `[]` represent an array

In [ ]:
# load nested file
nested <- fromJSON("https://raw.githubusercontent.com/khasenst/datasets_teaching/refs/heads/main/nested_json_example.json")

# print nested file
toJSON(nested, pretty = TRUE)

* In this format,
  * Activites are nested within "hobbies"
  * Address details are nested within "address"
  * All details are nested within "students"

* JSON format avoids the repetitive formatting of tabular representations of nested data

<br>

<br>

---

<br>

#### Loading JSON Data

* `R` loads JSON formatted data as unstructured lists [`list()`]

* We load JSON files using the `fromJSON()` function in the `jsonlite` library

In [ ]:
# load json data
data <- fromJSON("https://raw.githubusercontent.com/khasenst/datasets_teaching/refs/heads/main/nested_json_example.json")

<br>

* Remember, it is an unstructured list, so we can use the `$` operator to pull the nested data!

In [ ]:
# json data is loaded as an unstructured list
class(data)

<br>

* To view the contents of the unstructured list, we can use the `str()` and `names()` functions

In [ ]:
# view structure of json data in unstructured list format
str(data)

In [ ]:
# using the names() function
names(data)

In [ ]:
# using the names() function
names(data$students)

In [ ]:
# using the $ to extract fields from the unstructured list
data$students$name

In [ ]:
# pulling a field
data$students$hobbies

In [ ]:
data$students$address

<br>

* Depending on how the JSON data is structured, the `fromJSON()` function attempts to simplify our data into a dataframe

* This only occurs if subfields do not further contains lists as indicated by curly braces `{}`

<br>

* For example, the new JSON formatted dataset now includes the address details as additional entries, as opposed to starting another list

In [ ]:
# old format - address has a nested structure
'{
  "students": [
    {
      "name": "Alice",
      "age": 19,
      "address": {
        "street": "123 Main St",
        "city": "San Diego",
        "zipcode": "92182"
      },
      "hobbies": ["reading", "hiking", "surfing"]
    },
    {
      "name": "Pedro",
      "age": 18,
      "address": {
        "street": "456 Elm St",
        "city": "Los Angeles",
        "zipcode": "90745"
      },
      "hobbies": ["gaming", "cycling"]
    }
  ]
}'

In [ ]:
# new json format without additional nesting for address
json_string <-
'{

  "students": [
    {
      "name": "Alice",
      "age": 19,
      "address_street": "123 Main St",
      "address_city" : "San Diego",
      "address_zip" : "92182",
      "hobbies": ["reading", "hiking", "surfing"]
    },
    {
      "name": "Pedro",
      "age": 18,
      "address_street": "456 Elm St",
      "address_city": "Los Angeles",
      "address_zip": "90745",
      "hobbies": ["gaming", "cycling"]
    }
  ]

}'

In [ ]:
# load the string from the JSON format
data2 <- fromJSON(json_string)

In [ ]:
#  list output
str(data2)

In [ ]:
#  Now a data frame
data2$students

<br>

* We typically prefer the latter because the `fromJSON()` function does the work for us!

* If not, we must restructure our data using code to convert our data into a data frame

<br>

<br>

---

<br>

### Application Program Interfaces (APIs)

* Given the efficiency of the JSON format for storing nested data, many companies make their data available as JSON files

* The way in which they make their data available is through an application program interface (API)

<br>

* An API is a set of rules that allows different software applications to communicate with each other for data transfer

* We can use an API to download data from an institution's servers to our R workspace!

<br>

* An example is the API for the World Health Organization (WHO)

  https://apps.who.int/gho/data/node.resources.api

* Different APIs have different rules for extracting data

* Institutions typically provide documentation on how to access their data

* We will focus on the WHO API as an example

<br>

<br>

---

<br>

#### Extracting Data using the WHO API

* Available "variables" are listed here: https://ghoapi.azureedge.net/api/

* One of the variables is life expectancy (at birth) `WHOSIS_000001`. Let's take a look!

<br>

* Loading the data on that variable

  https://ghoapi.azureedge.net/api/WHOSIS_000001

In [ ]:
# path to data
url_path <- "https://ghoapi.azureedge.net/api/"

# selected variable
variable <- "WHOSIS_000001"

json_data <- fromJSON(paste0(url_path, variable),
                     #simplifyDataFrame = FALSE  # if you want it as a list, not a data frame
                     )

In [ ]:
# check contents of dataset
str(json_data)

In [ ]:
# check headers of dataset
names(json_data)

<br>

* For this particular API, the `value` list entry contains our data

* Thankfully, the `fromJSON` function was able to convert the JSON values into an `R` data frame...thank you WHO!

In [ ]:
# view head of data frame
head(json_data$value, 3)

# store data
data <- json_data$value

<br>

* Now that we have our dataset in a data frame in `R`, we can organize our data and do an analysis! We'll do this for our assignment.

<br>

<br>

---

<br>

#### Filtering/Subsetting via url

* The previous example shows how to import data in JSON format from an institutional website using their API

* But what if we don't want ALL of their data?
* What if their data is way too large for our purposes?


<br>

* Is there a way to select only what we need?
* Is there a way to download their data in pieces?
* Is there a way to subset on THEIR machines?

<br>

* The answer is Yes!

* Similar to filtering/subsetting in `R`, we can subset the data in the url itself

<br>

* For example, the script below downloads the same `WHOSIS_000001` dataset but with the following constraints

  * `SpatialDimType` must be `REGION`
  * `NumericValue` must not be missing (`NULL`)
  * `TimeDim` must be greater than or equal to `2020`



In [ ]:
# root path to API
url_path <- "https://ghoapi.azureedge.net/api/"

# selected variable
variable <- "WHOSIS_000001"

# filter query
filter1 <- "?$filter=SpatialDimType%20eq%20'REGION'"
filter2 <- "%20and%20NumericValue%20ne%20null"
filter3 <- "%20and%20TimeDim%20ge%202020"

In [ ]:
# query url
query_url <- paste0(url_path, variable, filter1, filter2, filter3)
query_url

In [ ]:
# import the data as an unstructured list
json_data <- fromJSON(query_url,
                     #simplifyDataFrame = FALSE  # if you want it as a list, not a data frame
                     )

<br>

* Viewing the imported data, we see that our filtering/subsetting requirements are met

In [ ]:
# the result
str(json_data)

<br>

* Instructions on how to do this are typically on the institution's website

* Note that in-depth API queries are outside the scope of this class

<br>

<br>

---

<br>

#### Exporting data as a JSON file

* We are able to export datasets as JSON files using
  * `toJSON()` - converts the data into a json class similar to character string
  * `write()` - then exports the character string using a given filename

<br>

* Let's do this with our initial example data

In [ ]:
# load nested file
json_data <- fromJSON("https://raw.githubusercontent.com/khasenst/datasets_teaching/refs/heads/main/nested_json_example.json")

str(json_data)

<br>

* We then convert the unstructured list into a JSON formatted string

In [ ]:
# convert to json string
json_data <- toJSON(json_data, pretty = TRUE)
json_data

In [ ]:
# class after toJSON()
class(json_data)

<br>

* Similar to `.csv` for comma separated value files, here, we use `.json` as the extension for JSON files

* The `write()` function is similar to the other exporting functions [`write.csv()`, `write.table()`], where you specify the data and the filepath

In [ ]:
# export the json data structure to Colab
write(json_data, "student_data.json")